In [1]:
import pandas
import numpy as np
import statistics as stats

#Imports para calcular círculo a partir de tres puntos
from sympy import Eq
from sympy.geometry import Point, Circle

#Imports para dibujar un circulo
import matplotlib.pyplot as plt

import random as rd


class Punto(): 
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def print(self):
        print('(',self.x, ', ',self.y,')')
    def __repr__(self):
        return '(%s, %s)' % (self.x,self.y)
    def __str__(self):
        return '(%s, %s)' % (self.x,self.y)
    
class Cluster(): 
    def __init__(self, x, y, radio):
        self.centro = Punto(x,y)
        self.radio = radio
        
    def print(self):
        print('Centro: ',self.centro, ' Radio: ',self.radio)
    def __repr__(self):
        return 'Centro: %s Radio: %s' % (self.centro,self.radio)
    def __str__(self):
        return 'Centro: %s Radio: %s' % (self.centro,self.radio)
    
    
    
    
def drawCircleAndPoints(Puntos, Clusters, color):  # learningaboutelectronics.com/Articles/How-to-draw-a-circle-using-matplotlib-in-Python.php
    
    Xs = []
    Ys = []
    for p in Puntos:
        Xs.append(p.x)
        Ys.append(p.y)
    plt.plot(Xs, Ys, 'ro', color=color)
    
    for cluster in Clusters:
        plt.plot(cluster.centro.x,cluster.centro.y, 'ro', color='red')
        circle = plt.Circle((cluster.centro.x, cluster.centro.y), cluster.radio, color=color, fill=False)
        ax=plt.gca()
        ax.add_patch(circle)
        plt.axis('scaled')
    
    plt.show()

In [2]:
def obtener_3_puntos_aleatorio(puntos):
    
    randomVals = rd.sample(puntos,3)
        
    return randomVals[0],randomVals[1],randomVals[2]

In [3]:
def clusters_iniciales_aleatorio(puntos,n_clusters, n_intentos):
    
    arrayClusters = []
    
    for i in range(n_intentos): 
        p = obtener_3_puntos_aleatorio(puntos)
        clusters = []

        for j in range(n_clusters):
            cluster = Cluster(p[j].x,p[j].y, rd.randint(1,10))
            clusters.append(cluster)
            
        arrayClusters.append(clusters)
    
    return arrayClusters

In [4]:
def get_cluster_casero(puntos):
    
    p1 = puntos[0]
    p2 = puntos[1]
    p3 = puntos[2]
    
    x1 = p1.x
    x2 = p2.x
    x3 = p3.x
    y1 = p1.y
    y2 = p2.y
    y3 = p3.y
    
    A = np.array([[x3-x1,y3-y1],[x3-x2,y3-y2]])
    Y = np.array([(x3**2 + y3**2 - x1**2 - y1**2),(x3**2+y3**2 - x2**2-y2**2)])
    if np.linalg.det(A) == 0:
        return False
    Ainv = np.linalg.inv(A)
    X = 0.5*np.dot(Ainv,Y)
    x,y = X[0],X[1]
    r = sqrt((x-x1)**2+(y-y1)**2)
    return (x,y),r

In [5]:
def get_cluster(puntos): # REF: https://www.geeksforgeeks.org/equation-of-circle-when-three-points-on-the-circle-are-given/
    
    p1 = puntos[0]
    p2 = puntos[1]
    p3 = puntos[2]
    
    print("################################# GET_CLUSTER ###################################")
    print("####### Puntos: #########")
    print(p1)
    print(p2)
    print(p3)
    
    circle = Circle(Point(p1.x,p1.y), Point(p2.x,p2.y), Point(p3.x,p3.y))
    
    print("#### CIRCLE: #######")
    print(circle)
    
    res = Cluster(float(circle.center.x),float(circle.center.y),float(circle.radius))
    
    return res

In [6]:
def grados_pertenencia(p,clusters):
    
    #Calculamos la distancia de el punto a cada cluster
    distancias = []
    
    for i in range(len(clusters)):
        d = np.sqrt(pow(p.x - clusters[i].centro.x,2) + pow(p.y - clusters[i].centro.y,2))
        distancias.append(abs(d-clusters[i].radio))
    
    #Calculamos el grado de pertencia a cada cluster
    pertenencias = []
    
    for i in range(len(clusters)):
        if(distancias[i]==0.0):
            per = 100.0
        else:
            per = 100/(pow(distancias[i],2))
        pertenencias.append(per)
        
    #Normalizamos
    suma = sum(pertenencias)
    pertenencias = np.divide(pertenencias,suma)
    
    return pertenencias

In [7]:
#Le pasamos los 3 puntos con mayor grado de pertenencia a un cluster:
def puntos_mayor_grado_por_cluster(puntos,cluster,n_cluster):
    per = []
    res = []

    for i in range(len(puntos)):
        per.append(grados_pertenencia(puntos[i],cluster)[n_cluster])
    
    res.append(puntos[per.index(max(per))])
    puntos = np.delete(puntos,per.index(max(per)),0)
    per.remove(max(per))
    
    res.append(puntos[per.index(max(per))])
    puntos = np.delete(puntos,per.index(max(per)),0)
    per.remove(max(per))
    
    res.append(puntos[per.index(max(per))])
    
    return res

In [8]:
def get_real_clusters(puntos,clusters):
    res = []
    
    for i in range(len(clusters)):
        pts_pertenencia = puntos_mayor_grado_por_cluster(puntos,clusters,i)
        #print("GET_REAL_CLUSTER")
        #print(pts_pertenencia)
        res.append(get_cluster(pts_pertenencia))
        
    return res 

In [9]:
def iterar_get_real_cluster(puntos,prev_clusters,razon, max_iteraciones, imprimir):
    
    iterar = 1
    iteraciones = 0;
    
    while(iterar and iteraciones <= max_iteraciones):
        iteraciones = iteraciones + 1
        nuevo_factor = []
        #print("ITERAR_GET_REAL_CLUSTER")
        #print(puntos)
        #print(prev_clusters)
        next_clusters = get_real_clusters(puntos,prev_clusters)
        if(imprimir):
            print(next_clusters)
        for i in range(len(prev_clusters)):
            nuevo_factor.append(np.sqrt((prev_clusters[i].centro.x - next_clusters[i].centro.x)**2 + (prev_clusters[i].centro.y - next_clusters[i].centro.y)**2) + abs(prev_clusters[i].radio - next_clusters[i].radio))
            if(imprimir):
                print(nuevo_factor)
        if(max(nuevo_factor) < razon):    
            iterar = 0
        prev_clusters = next_clusters
    return next_clusters

In [10]:
def iterar_iterar_get_real_cluster(arrayClusters,puntos,razon, max_iteraciones, imprimir):
    
    for i in range(len(arrayClusters)):
        print(i)
        cluster = iterar_get_real_cluster(puntos,arrayClusters[i],razon,max_iteraciones,imprimir)
        
    return cluster

In [ ]:
#Agustin 2+:

#Todos los puntos:
puntoscsv = pandas.read_csv('Data/puntos_agustin_2.csv', header=None, names=['x', 'y']).values
puntos = []
for i in range(len(puntoscsv)):
    puntos.append(Punto(puntoscsv[i][0],puntoscsv[i][1]))
puntos

#Clusters iniciales:
arrayClusters = clusters_iniciales_aleatorio(puntos,3,10)



clusters = iterar_iterar_get_real_cluster(arrayClusters, puntos,0.000002,100,0)
clusters

drawCircleAndPoints(puntos,clusters,'blue')

In [11]:
puntos = []
puntos.append(Point(11.6,6.6))
puntos.append(Point(11.2,7.0))
puntos.append(Point(11.0,7.2))
get_cluster_casero(puntos)

TypeError: No loop matching the specified signature and casting
was found for ufunc det

In [13]:
puntos = []
puntos.append(Point(0,0))
puntos.append(Point(2,2))
puntos.append(Point(4,4))
get_cluster_casero(puntos)

TypeError: No loop matching the specified signature and casting
was found for ufunc det

In [ ]:
circle = Circle(Point(11.6,6.6), Point(11.2,7.0), Point(11.0,7.2))
circle